<a href="https://colab.research.google.com/github/Siri-cod/LLMcourse/blob/main/tutorials/08a-mechanistic-interpretability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sheet 8.1: Mechanistic interpretability
==========
**Author**: Polina Tsvilodub

One criticism often raised in context of LLMs is their blackbox nature, i.e., the inscrutability of the mechanics of the models and how or why they arrive at predictions, given the input.
We have seen some methods that help to map out how models behave on certain tasks ([sheet 7.1](https://cogsciprag.github.io/Understanding-LLMs-course/tutorials/07a-behavioral-assessment.html)), what aspects of the input critically affect the output and which information the models process ([sheet 6.1](https://cogsciprag.github.io/Understanding-LLMs-course/tutorials/06a-attribution.html)).
In this sheet, we will look at methods for identifying the *computational mechanisms* that lead to the outputs, i.e., at mechanistic interpretability. It can be seen as trying to reverse-engineer the computational algorithms the mdoel has learned during training and that are active during certain tasks.

## Early decoding

First, we will look at early decoding, i.e., at applying the "unembedding" layer (projecting hidden representations into vocabulary space by applying a linear and a softmax layer) to representations in layers throughout the model (not just the last layer). For this, we will need to output results of the calculations in the layers.
Furthermore, it is helpful to be able to access different weights of the model, which we did in [sheet 3.1](https://cogsciprag.github.io/Understanding-LLMs-course/tutorials/03a-tokenization-transformers.html).

First, we will look at logit lens application through a neat package [`nnsight`](https://nnsight.net/notebooks/tutorials/logit_lens/), and then at a "manual" implementation example.

In [2]:
!pip install nnsight

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from IPython.display import clear_output
from nnsight import LanguageModel
from typing import List, Callable
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import plotly.express as px
import plotly.io as pio
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
import numpy as np
from torch import nn

pio.renderers.default = "colab"

In [4]:
# Load gpt2
model = LanguageModel("openai-community/gpt2", device_map="auto", dispatch=True)
print(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (generator): Generator(
    (streamer): Streamer()
  )
)


In [9]:
# define the text for which we want to see the early decoded tokens
prompt= "The Eiffel Tower is in the city of"
layers = model.transformer.h
probs_layers = []

with model.trace() as tracer:
    with tracer.invoke(prompt) as invoker:
        for layer_idx, layer in enumerate(layers):
            # Process layer output through the model's head and layer normalization
            layer_output = model.lm_head(model.transformer.ln_f(layer.output[0]))

            # Apply softmax to obtain probabilities and save the result
            probs = torch.nn.functional.softmax(layer_output, dim=-1).save()
            probs_layers.append(probs)

probs = torch.cat([probs.value for probs in probs_layers])

# Find the maximum probability and corresponding tokens for each position
max_probs, tokens = probs.max(dim=-1)

# Decode token IDs to words for each layer
words = [[model.tokenizer.decode(t.cpu()).encode("unicode_escape").decode() for t in layer_tokens]
    for layer_tokens in tokens]

# Access the 'input_ids' attribute of the invoker object to get the input words
input_words = [model.tokenizer.decode(t) for t in invoker.inputs[0][0]["input_ids"][0]]

In [10]:
fig = px.imshow(
    max_probs.detach().cpu().numpy(),
    x=input_words,
    y=list(range(len(words))),
    color_continuous_scale=px.colors.diverging.RdYlBu_r,
    color_continuous_midpoint=0.50,
    text_auto=True,
    labels=dict(x="Input Tokens", y="Layers", color="Probability")
)

fig.update_layout(
    title='Logit Lens Visualization',
    xaxis_tickangle=0
)

fig.update_traces(text=words, texttemplate="%{text}")
fig.show()

Below, you can see an explicit implementation of early decoding, show-casing how to access and track hidden representations at different layers and how to apply the language modeling head (i.e., unembedding layer) to them. Accessing anf tracking the hidden representations is also useful for interpretability techniques we will see below -- patching those representations between runs.

In [6]:
def get_device():
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    return device

def load_gpt2(version):
    device= get_device()
    tokenizer = AutoTokenizer.from_pretrained(version)
    model = AutoModelForCausalLM.from_pretrained(version, torch_dtype=torch.float16).to(device)
    return model, tokenizer

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
    def forward(self, x):
        return self.lambd(x)

class ModelWrapper(nn.Module):

    def __init__(self, model, tokenizer):
        super().__init__()
        self.model = model.eval()
        self.model.activations_ = {}
        self.tokenizer = tokenizer
        self.device = get_device()
        self.num_layers = len(self.model.transformer.h)
        self.hooks  = []
        self.layer_pasts = {}

    def tokenize(self, s):
        return self.tokenizer.encode(s, return_tensors='pt').to(self.device)

    def list_decode(self, inpids):
        return [self.tokenizer.decode(s) for s in inpids]

    def layer_decode(self, hidden_states):
        raise Exception("Layer decode has to be implemented!")

    def get_layers(self, tokens, **kwargs):
        outputs = self.model(input_ids=tokens, output_hidden_states=True, **kwargs)
        hidden_states, true_logits = outputs.hidden_states, outputs.logits
        logits = self.layer_decode(hidden_states)
        #logits[-1] = true_logits.squeeze(0)[-1].unsqueeze(-1) #we used to just replace the last logits because we were applying ln_f twice
        return torch.stack(logits).squeeze(-1)#, true_logits.squeeze(0)

    def get_layers_w_attns(self, tokens, **kwargs):
        outputs = self.model(input_ids=tokens, output_hidden_states=True, output_attentions=True, **kwargs)
        hidden_states, true_logits = outputs.hidden_states, outputs.logits
        logits = self.layer_decode(hidden_states)
        #logits[-1] = true_logits.squeeze(0)[-1].unsqueeze(-1)
        return torch.stack(logits).squeeze(-1), outputs.attentions#, true_logits.squeeze(0)

    def rr_per_layer(self, logits, answer, debug=False):
        #reciprocal rank of the answer at each layer
        answer_id = self.tokenizer.encode(answer)[0]
        if debug:
            print("Answer id", answer_id, answer)

        rrs = []
        for i,layer in enumerate(logits):
            soft = F.softmax(layer,dim=-1)
            sorted_probs = soft.argsort(descending=True)
            rank = float(np.where(sorted_probs.cpu().numpy()==answer_id)[0][0])
            rrs.append(1/(rank+1))

        return np.array(rrs)

    def prob_of_answer(self, logits, answer, debug=False):
        answer_id = self.tokenizer.encode(answer)[0]
        if debug:
            print("Answer id", answer_id, answer)
        answer_probs = []
        first_top = -1
        mrrs = []
        for i,layer in enumerate(logits):
            soft = F.softmax(layer,dim=-1)
            answer_prob = soft[answer_id].item()
            sorted_probs = soft.argsort(descending=True)
            if debug:
                print(f"{i}::", answer_prob)
            answer_probs.append(answer_prob)
        #is_top_at_end = sorted_probs[0] == answer_id
        return np.array(answer_probs)

    def print_top(self, logits, k=10):
        for i,layer in enumerate(logits):
            print(f"{i}", self.tokenizer.decode(F.softmax(layer,dim=-1).argsort(descending=True)[:k]) )

    def topk_per_layer(self, logits, k=10):
        topk = []
        for i,layer in enumerate(logits):
            topk.append([self.tokenizer.decode(s) for s in F.softmax(layer,dim=-1).argsort(descending=True)[:k]])
        return topk

    def get_activation(self, name):
        #https://github.com/mega002/lm-debugger/blob/01ba7413b3c671af08bc1c315e9cc64f9f4abee2/flask_server/req_res_oop.py#L57
        def hook(module, input, output):
            if "in_sln" in name:
                num_tokens = list(input[0].size())[1]
                self.model.activations_[name] = input[0][:, num_tokens - 1].detach()
            elif "mlp" in name or "attn" in name or "m_coef" in name:
                if "attn" in name:
                    num_tokens = list(output[0].size())[1]
                    self.model.activations_[name] = output[0][:, num_tokens - 1].detach()
                    self.model.activations_['in_'+name] = input[0][:, num_tokens - 1].detach()
                elif "mlp" in name:
                    num_tokens = list(output[0].size())[0]  # [num_tokens, 3072] for values;
                    self.model.activations_[name] = output[0][num_tokens - 1].detach()
                elif "m_coef" in name:
                    num_tokens = list(input[0].size())[1]  # (batch, sequence, hidden_state)
                    self.model.activations_[name] = input[0][:, num_tokens - 1].detach()
            elif "residual" in name or "embedding" in name:
                num_tokens = list(input[0].size())[1]  # (batch, sequence, hidden_state)
                if name == "layer_residual_" + str(self.num_layers-1):
                    self.model.activations_[name] = self.model.activations_[
                                                        "intermediate_residual_" + str(final_layer)] + \
                                                    self.model.activations_["mlp_" + str(final_layer)]

                else:
                    if 'out' in name:
                        self.model.activations_[name] = output[0][num_tokens-1].detach()
                    else:
                        self.model.activations_[name] = input[0][:,
                                                            num_tokens - 1].detach()

        return hook

    def reset_activations(self):
        self.model.activations_ = {}


class GPT2Wrapper(ModelWrapper):

    def layer_decode(self, hidden_states):
        logits = []
        for i,h in enumerate(hidden_states):
            h=h[:, -1, :] #(batch, num tokens, embedding size) take the last token
            if i == len(hidden_states)-1:
                normed = h #ln_f would already have been applied
            else:
                normed = self.model.transformer.ln_f(h)
            l = torch.matmul(self.model.lm_head.weight, normed.T)
            logits.append(l)
        return logits

    def add_hooks(self):
        for i in range(self.num_layers):
            #intermediate residual between
            #print('saving hook')
            self.hooks.append(self.model.transformer.h[i].ln_1.register_forward_hook(self.get_activation(f'in_sln_{i}')))
            self.hooks.append(self.model.transformer.h[i].attn.register_forward_hook(self.get_activation('attn_'+str(i))))
            self.hooks.append(self.model.transformer.h[i].ln_2.register_forward_hook(self.get_activation("intermediate_residual_" + str(i))))
            self.hooks.append(self.model.transformer.h[i].ln_2.register_forward_hook(self.get_activation("out_intermediate_residual_" + str(i))))
            self.hooks.append(self.model.transformer.h[i].mlp.register_forward_hook(self.get_activation('mlp_'+str(i))))
            #print(self.model.activations_)


    def get_pre_wo_activation(self, name):
        #wo refers to the output matrix in attention layers. The last linear layer in the attention calculation

        def hook(module, input, output):
            #use_cache=True (default) and output_attentions=True have to have been passed to the forward for this to work
            _, past_key_value, attn_weights = output
            value = past_key_value[1]
            pre_wo_attn = torch.matmul(attn_weights, value)
            self.model.activations_[name]=pre_wo_attn

        return hook

    def get_past_layer(self, name):
        #wo refers to the output matrix in attention layers. The last linear layer in the attention calculation

        def hook(module, input, output):
            #use_cache=True (default) and output_attentions=True have to have been passed to the forward for this to work
            #print(len(output), output, name)
            _, past_key_value, attn_weights = output
            self.layer_pasts[name]=past_key_value

        return hook

    def add_mid_attn_hooks(self):
        for i in range(self.num_layers):
            self.hooks.append(self.model.transformer.h[i].attn.register_forward_hook(self.get_pre_wo_activation('mid_attn_'+str(i))))

            self.hooks.append(self.model.transformer.h[i].attn.register_forward_hook(self.get_past_layer('past_layer_'+str(i))))

    def rm_hooks(self):
        for hook in self.hooks:
            hook.remove()

    def reset_activations(self):
        self.activations_ = {}
        self.last_pasts = {}


In [7]:
model, tokenizer = load_gpt2('gpt2-medium')
model = model.float()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
wrapper = GPT2Wrapper(model, tokenizer)
def tokenize(text):
    inp_ids = wrapper.tokenize(text)
    str_toks = wrapper.list_decode(inp_ids[0])
    return inp_ids, str_toks

In [9]:
poland_text="""Q: What is the capital of France?
A: Paris
Q: What is the capital of Poland?
A:"""

poland_ids, pol_toks = tokenize(poland_text)

In [10]:
poland_ids.shape

torch.Size([1, 26])

In [11]:
logits = wrapper.get_layers(poland_ids)
wrapper.print_top(logits[1:]) #skip the embedding layer

0  ( [ The:,
 at and Act A
1  A The ( [ Is59
 At and40
2  A [ ( The At Is Act at59,
3  A [ ( Act At Is The CH An at
4  A [ At Q (Q The Are M An
5  A M No At The payable Q Qu (Q
6  No M A The C Die An H En Qu
7  C A No The M n P N H An
8  A The C P H No n Ass N T
9  A C No nil The Ch P An H N
10  A The G C N P No Me An Le
11  A C N None P G The Pr Ce H
12  Unknown None C G A N Bar The Ch P
13  C P N G B A Unknown St None The
14  St N G P Poland B C Pol A D
15  Poland P St Pol Warsaw Polish N B G Germany
16  Poland Warsaw Polish Poles Budapest Prague Pol Germany Berlin Moscow
17  Poland Warsaw Polish Poles Budapest Prague � Pol Lithuania Moscow
18  Poland Warsaw Polish Prague Budapest Poles Moscow � Berlin Kiev
19  Warsaw Poland Polish Budapest Prague Moscow Berlin Kiev � Frankfurt
20  Warsaw Poland Prague Budapest Polish Moscow Kiev Berlin Frankfurt Brussels
21  Warsaw Poland Polish Prague Budapest � Kiev Sz Berlin Moscow
22  Warsaw Poland Prague Budapest K W Kiev Sz Moscow Berlin
23  W

In [12]:
# look at the shape of the logits and try to understand what they represent
logits.shape

torch.Size([25, 50257])

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 8.1.1: Early decoding</span></strong>
>
> 1. (For yourself) Read through the code above and make sure that you understand what it does and why. Make sure to understand the concept of early decoding.
> 2. Try a different example (maybe for a different task). Do you see a similar pattern (comparing results between the layers)?


## Residual stream

Next, the lecture discussed the role of the *residual stream*, i.e., the "stream" which passes information between the transformer blocks and, only undergoes linear transformations. Note that there is no block or layers called "residual stream" in the transformer architecture (i.e., if you were to print a pre-defined architecture); rather, it is a conceptual interpretation of the transformer architecture, realising that due to residual connections within the transformer blocks, one can look at the flow of information (i.e., the vector representations) as being *read from* by the transformer blocks (i.e., reading results of previous computations), and then *writing* the results of the attention calculations back to the representations (via linear operations applied to previous representations).

Below is a small example of accessing the residual stream. The idea behind this analysis is understanding what exactly the mechanistic role of the single computations (i.e., applying attention vs. the FFNN layer).
The code below works with so-called hooks, i.e., functions that are "hooked onto" the forward pass of through the model and are executed together with the normal computations, when the model is called. They are based on [this](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.register_forward_hook) native PyTorch functionality. Their implementation is in the code cells above.

Specicifically, the question tested in the code below is which computational step promotes "Warsaw" over "Poland", specifically, whether it is the pass through the FFNN. This is done by using the same early decoding technique and looking at the model's prediction before and after applying the FFNN.

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 8.1.2: Residual stream decoding</span></strong>
>
> 1. (For yourself) Read through the code above and make sure that you understand what it does and why. How is the question above operationalized?
> 2. Inspecting the results, would you say that the FFNN is responsinble for prompting "Warsaw"?
> 3. In case you saw a similar pattern for your other example above, try to adapt the code below to your example.
> 4. [Optional] If you are curious to see more, read the excellent paper above and look at the full [demo notebook](https://github.com/jmerullo/lm_vector_arithmetic/blob/main/demo.ipynb).

In [13]:
"""
We can decode at the residual stream between the attention and FFN to show that
it is the FFN update that updates from Poland to Warsaw, but a slightly easier way to do this is to
subtract the FFN update that was applied at layer 19 to show the intermediate residual stream state
(i.e., between the attention and FFN)
"""

wrapper.add_hooks()
out = wrapper.model(input_ids = poland_ids, output_hidden_states=True) #run it again to activate hooks
logits = out.logits
hidden_states = out.hidden_states
hidden_states = list(hidden_states)[1:] #skip the embedding layer to stay consistent with our indexing

#get the FFN output update at layer 19
o_city = wrapper.model.activations_['mlp_19']


print(len(hidden_states)) #24

layer_logits = wrapper.layer_decode(hidden_states)
layer_logits = torch.stack(layer_logits).squeeze(-1)
print("Original top tokens at layer 19")
wrapper.print_top(layer_logits[19].unsqueeze(0))

hidden_states[19]-=o_city


layer_logits = wrapper.layer_decode(hidden_states)
layer_logits = torch.stack(layer_logits).squeeze(-1)
print("After subtracting mlp_19 (o_city)")
wrapper.print_top(layer_logits[19].unsqueeze(0))


24
Original top tokens at layer 19
0  Warsaw Poland Polish Budapest Prague Moscow Berlin Kiev � Frankfurt
After subtracting mlp_19 (o_city)
0  Poland Warsaw Polish Budapest Prague Poles � Moscow Berlin Kiev


## Activation patching

Another technique discussed in the lecture is *activation patching*. This can be seen as a method for *causal intervention* on the computational mechanisms of the model. For instance, certain results of the computations are injected, and the effect these results have on the outcome are observed. We could, of course, inject random values, but that might require trying quite many values before being able to observe meaningful differences in the output. Therefore, instead, certain representations computed in one pass through the model are taken (i.e., representations from the *clean run*) and injected into another (the *corrupted run*). These injected representations are called the patch.

Patching techniques are quite tricky to work with since patching something in an intermediate layer affects not only the local layer, but also all the downstream computations. To be able to retrieve meaningful results, careful comparison and, e.g., freezing of subsequent representations is required.

Below is an example of using activation patching on the Indirect Object Identification task, with the help of the library [`transformer_lens`](https://github.com/TransformerLensOrg/TransformerLens).
The idea of the task if simple: given a sentence like "After John and Mary went to the store, Mary gave a bottle of milk to", identify the indirect object (i.e., "John", rather than "Mary"). The goal is to identify which model activations (i.e., representation computed within the transformer ) are important for completing a task. We do this by setting up a clean prompt and a corrupted prompt, for instance:

Clean prompt: After John and Mary went to the store, Mary gave a bottle of milk to

Corrupted prompt: After John and Mary went to the store, John gave a bottle of milk to

Further, we define a metric for performance on the task, e.g., the difference in logits of the correct token, given different inputs (recall our methods for assessing model performance, where one core assumption is that a model can perform a task well if it assigns higher log probabilities (i.e., the logits are higher) for correct predictions that for incorrect ones).

We then pick a specific model activation, run the model on the corrupted prompt, but then intervene on that activation and patch in its value when run on the clean prompt. We then apply the metric, and see how much this patch has recovered the clean performance.
Essentially, we ask: given a corrupted input, if we inject certain representations from the "correct" run, can we "fix" the performance? If we can, we know that the injected representations (causally!) contribute to producing the correct output.

The code below is taken from [this](https://github.com/TransformerLensOrg/TransformerLens/blob/main/demos/Main_Demo.ipynb) demo.

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 8.1.3: Activation patching</span></strong>
>
> 1. (For yourself) Read through the code above and make sure that you understand what it does and why.
>
> 2. Try a different pair os a clean and corrupt sentence. What happens?

In [14]:
#!pip install transformer_lens plotly

In [15]:
from transformer_lens import HookedTransformer
import plotly.express as px
import transformer_lens.utils as utils
import tqdm
from functools import partial
import torch

ModuleNotFoundError: No module named 'transformer_lens'

In [ ]:
# load the model within the wrapper of the library which allows to easily access and patch activations

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

In [ ]:
# first, we check if the model can do the task at all
# i.e., we compare the difference in logits for the correct and incorrect answer
# given different inputs without any interventions

clean_prompt = "After John and Mary went to the store, Mary gave a bottle of milk to"
corrupted_prompt = "After John and Mary went to the store, John gave a bottle of milk to"

clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

def logits_to_logit_diff(logits, correct_answer=" John", incorrect_answer=" Mary"):
    # model.to_single_token maps a string value of a single token to the token index for that token
    # If the string is not a single token, it raises an error.
    correct_index = model.to_single_token(correct_answer)
    incorrect_index = model.to_single_token(incorrect_answer)
    return logits[0, -1, correct_index] - logits[0, -1, incorrect_index]

# We run on the clean prompt with the cache so we store activations to patch in later.
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = logits_to_logit_diff(clean_logits)
print(f"Clean logit difference: {clean_logit_diff.item():.3f}")

# We don't need to cache on the corrupted prompt.
corrupted_logits = model(corrupted_tokens)
corrupted_logit_diff = logits_to_logit_diff(corrupted_logits)
print(f"Corrupted logit difference: {corrupted_logit_diff.item():.3f}")

In [ ]:
# define a helper

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)


In [ ]:
# We define a residual stream patching hook
# We choose to act on the residual stream at the start of the layer, so we call it resid_pre
# The type annotations are a guide to the reader and are not necessary
def residual_stream_patching_hook(
    resid_pre,
    hook,
    position
):
    # Each HookPoint has a name attribute giving the name of the hook.
    clean_resid_pre = clean_cache[hook.name]
    # NOTE: this is the key step in the patching process
    # where we replace the activations in the residual stream with the same activations from the clean run
    resid_pre[:, position, :] = clean_resid_pre[:, position, :]
    return resid_pre

# We make a tensor to store the results for each patching run.
# We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
num_positions = len(clean_tokens[0])
ioi_patching_result = torch.zeros((model.cfg.n_layers, num_positions), device=model.cfg.device)

for layer in tqdm.tqdm(range(model.cfg.n_layers)):
    for position in range(num_positions):
        # Use functools.partial to create a temporary hook function with the position fixed
        temp_hook_fn = partial(residual_stream_patching_hook, position=position)
        # Run the model with the patching hook
        patched_logits = model.run_with_hooks(corrupted_tokens, fwd_hooks=[
            (utils.get_act_name("resid_pre", layer), temp_hook_fn)
        ])
        # Calculate the logit difference
        patched_logit_diff = logits_to_logit_diff(patched_logits).detach()
        # Store the result, normalizing by the clean and corrupted logit difference so it's between 0 and 1 (ish)
        ioi_patching_result[layer, position] = (patched_logit_diff - corrupted_logit_diff)/(clean_logit_diff - corrupted_logit_diff)



In [ ]:
# Add the index to the end of the label, because plotly doesn't like duplicate labels
token_labels = [f"{token}_{index}" for index, token in enumerate(model.to_str_tokens(clean_tokens))]
imshow(ioi_patching_result, x=token_labels, xaxis="Position", yaxis="Layer", title="Normalized Logit Difference After Patching Residual Stream on the IOI Task")


## Sparse Autoencoders (SAEs)

Finally, the lecture introduced sparse autoencoders as a tool for understanding representations within the LMs. The idea behind SAEs is to train an autoencoder that has a higher-diemnsional representational space (e.g., 200k) than the size of the model's hidden representation (e.g., 768-dimensional vectors). Because the LM space is lower dimensional, the hiddenrepresentations usually encode different information within one neuron (i.e., one dimension). The goal of training an SAE with "more space" on these representations allows to disentangle the information more and make it more interpretable.

As for other state-of-the-art techniques, there are libraries for traiing SAEs on LM representations, and even pretrained SAEs for different LLMs available on GitHub. One such library is [`sae-lens`](https://jbloomaus.github.io/SAELens/latest/).

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 8.1.4: Sparse autoencoders</span></strong>
>
> 1. (For yourself) [Here](https://colab.research.google.com/github/jbloomAus/SAELens/blob/main/tutorials/training_a_sparse_autoencoder.ipynb) is a notebook (in progress) with an example of training an SAE for the `tiny-stories-1L-21M` model. Look at the code and try to find the hidden dimension of the SAE. By which factor is it larger than the LM's hidden size?
>
> 2. (Optional) [Here](https://adamkarvonen.github.io/machine_learning/2024/06/11/sae-intuitions.html) is (one of many) blogpost elaborating on SAEs for LMs more.